# Cloud Storage

## Amazon S3 Data Warehousing

In [1]:
import os
import sys
import operator
import functools
import itertools
import boto
import warnings

from os.path import join as jp

from configparser import ConfigParser

LoadError: os not found
while loading In[1], in expression starting on line 1

## S3 Remote Data Warehousing

In [ ]:
boto.

In [2]:
def aws_config(cfg):
    """
    Queries local environment for aws configurations
    """
    home, user = os.getenv('HOME'), os.getlogin()
    valid_user = user in cfg.sections()

    return user if valid_user else cfg.sections()[0]

def s3_signin(**auth):
    """
    Convenience function for validating keys  and providing
    access to bucket shares.
    
    Returns S3Object
    """
    token_ids  = 'aws_access_key_id', 'aws_secret_access_key'

    cfg = ConfigParser()
    cfg.read(jp(os.getenv('HOME'), '.aws', 'credentials'))

    account    = itertools.repeat(aws_config(cfg), 2)
    valid_auth = all(auth.has_key(i) for i in token_ids)
    token      = zip(account, token_ids) if not valid_auth else [token_ids]
    store      = cfg if not auth else auth
    
    user_id    = dict(zip(token_ids, map(lambda t: store.get(*t), token)))
    
    if not all(user_id.values()):
        raise ValueError('No valid authorization found')

    return boto.connect_s3(**user_id)

LoadError: syntax: extra token "aws_config" after end of expression
while loading In[2], in expression starting on line 1

## Key and Configuration Management

In [3]:
s3 = s3_signin()

gl.aws.set_credentials(s3.gs_access_key_id, s3.gs_secret_access_key)

LoadError: s3_signin not defined
while loading In[3], in expression starting on line 1

## Streaming Remote IO

In [4]:
def remote_json_loader(filename):
    """
    Load JSON from a remote data store.
    """
    sf = gl.SFrame.read_csv(filename, delimiter='\n', header=False)
    return sf.unpack('X1', column_name_prefix='')

def gen_data_url(s3, bucket , dataset):
    s3_dir   = s3.get_bucket(bucket)
    s3_urls  = [
        '/'.join(['s3:/', s3_dir.name, d.name])
                for d in s3_dir.list(dataset)
    ]
    for url in s3_urls:
        yield url

def flatten(sf):
    """
    Flatten nested SFrame DataStructure. 
    """
    dtypes = dict(zip(sf.column_names(), gl.SFrame.dtype(sf)))
    d = {k:v for k,v in dtypes.items() if v in [dict, list]}
    sv = gl.SFrame.copy(sf)
    for p in d:
        sv = sv.unpack(p, column_name_prefix='')
    return sv

# This is Going To Take a While

In [5]:
gl.canvas.set_target('ipynb')

s3_bin = list()
yelp   = gen_data_url(s3, 'ds3-machine-learning', 'yelp')

for data in yelp:
    sf = flatten(remote_json_loader(data))
    sf.show()
    s3_bin.append(sf)

[INFO] This trial license of GraphLab Create is assigned to jjangsangy@gmail.com and will expire on June 09, 2015. Please contact trial@dato.com for licensing options or to request a free non-commercial license for personal or academic use.

[INFO] Start server at: ipc:///tmp/graphlab_server-57216 - Server binary: /usr/local/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1433586756.log
[INFO] GraphLab Server Version: 1.4.0


PROGRESS: Completed 1 of 7 part(s) with 1 file(s) remaining
PROGRESS: Completed 2 of 7 part(s) with 1 file(s) remaining
PROGRESS: Completed 3 of 7 part(s) with 1 file(s) remaining
PROGRESS: Completed 4 of 7 part(s) with 1 file(s) remaining
PROGRESS: Completed 5 of 7 part(s) with 1 file(s) remaining
PROGRESS: Completed 6 of 7 part(s) with 1 file(s) remaining
PROGRESS: Completed 7 of 7 part(s) with 1 file(s) remaining
PROGRESS: download: s3://ds3-machine-learning/yelp/yelp_academic_dataset_business.json to ../../var/tmp/graphlab-jjangsangy/57216/000000
PROGRESS:
PROGRESS: Finished parsing file s3://ds3-machine-learning/yelp/yelp_academic_dataset_business.json
PROGRESS: Parsing completed. Parsed 100 lines in 6.45642 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
----

In [22]:
a0 = s3_bin[0]
a1 = s3_bin[1]
a2 = s3_bin[2]
a3 = s3_bin[3]

In [25]:
a1.show()

<IPython.core.display.Javascript object>

In [17]:
a2.show()

<IPython.core.display.Javascript object>

In [19]:
a3.show()

<IPython.core.display.Javascript object>

In [24]:
a0.show()

<IPython.core.display.Javascript object>